In [1]:
pip install --upgrade google-genai transformers soundfile accelerate sentencepiece

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:

from google import genai
from google.genai import types
import base64

from google import genai

f = open("keys/untitled.txt")
key = f.read()

client = genai.Client(api_key=key)


def encode_image(path):
    with open(path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")
    return encoded

# Path to your input image
image_path = "images/dog.jpg"
encoded_data = encode_image(image_path)

system_prompt = (
    "You are a helpful AI Assistant. "
    "Given an image perform object detection and provide a text output "
    "which contains the information about the labels detected and their counts."
)

#response = client.models.generate_content(
 #   model="gemini-3-flash-preview",
  #  contents=[
   #     {"text":system_prompt},
   #     {"image":{"base64":image_b64}}
   # ],
 #   config=types.GenerateContentConfig(
  #      temperature=0.2
  #  )
#)

from google import genai
from google.genai import types

with open('images/dog.JPG', 'rb') as f:
  image_bytes = f.read()

context = [
    types.Part.from_bytes(
        data=image_bytes,
        mime_type='image/jpeg',
    ),
]

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=context
)

caption_text = response.text
print("RAW CAPTION:\n", caption_text)

RAW CAPTION:
 On the porch, there is a red road bicycle sitting next to a large dog with black, tan, and white fur. The bicycle features drop handlebars with black tape, thin road tires on silver rims, and a red frame with black and white accents. Behind the dog and bike, the porch has a yellow painted wall and white wooden floorboards. In the background, beyond the porch and through some green trees, a white pickup truck with a canopy and a red scooter are visible on the street.


In [11]:

from transformers import AutoProcessor, AutoModel
import numpy as np, soundfile as sf, torch, IPython.display as ipd

processor = AutoProcessor.from_pretrained("suno/bark-small")
model = AutoModel.from_pretrained("suno/bark-small")


Loading weights:   0%|          | 0/542 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.1.weight to fine_acoustics.lm_heads.0.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.2.weight to fine_acoustics.lm_heads.1.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_layers.3.weight to fine_acoustics.lm_heads.2.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie fine_acoustics.input_embeds_l

In [12]:

inputs = processor(text=caption_text, return_tensors="pt")

with torch.no_grad():
    audio = model.generate(**inputs)

audio_np = audio.cpu().numpy().flatten().astype(np.float32)
audio_np = audio_np / (np.max(np.abs(audio_np)) + 1e-9)

sf.write("Approach2_output.wav", audio_np, 16000, subtype="PCM_16")
ipd.Audio("Approach2_output.wav")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
Both `max_new_tokens` (=768) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=60) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=60) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both